In [1]:
from typing import Union,List

from rich import print as pprint

In [2]:
from quantumion.interface.analog import *
from quantumion.interface.math import *

from quantumion.compiler.analog import *
from quantumion.compiler.math import *
from quantumion.compiler.visitor import Visitor

# Math

In [3]:
expr = MathNum(value=1) + MathImag() - MathVar(name="a") * MathStr(string="sin(1)") / MathNum(value=2) ** MathVar(name="b")

pprint(expr)
pprint(expr.accept(PrintMathExpr()))

MathSub(
    class_='MathSub',
    expr1=MathAdd(class_='MathAdd', expr1=MathNum(class_='MathNum', value=1), expr2=MathImag(class_='MathImag')),
    expr2=MathDiv(
        class_='MathDiv',
        expr1=MathMul(
            class_='MathMul',
            expr1=MathVar(class_='MathVar', name='a'),
            expr2=MathFunc(class_='MathFunc', func='sin', expr=MathNum(class_='MathNum', value=1))
        ),
        expr2=MathPow(
            class_='MathPow',
            expr1=MathNum(class_='MathNum', value=2),
            expr2=MathVar(class_='MathVar', name='b')
        )
    )
)

1 + 1j - a * sin(1) / 2 ** b

## Distribute

In [4]:
expr = MathStr(string="(1+2)*3")

pprint(expr)
pprint(expr.accept(PrintMathExpr()))

MathMul(
    class_='MathMul',
    expr1=MathAdd(
        class_='MathAdd',
        expr1=MathNum(class_='MathNum', value=1),
        expr2=MathNum(class_='MathNum', value=2)
    ),
    expr2=MathNum(class_='MathNum', value=3)
)

(1 + 2) * 3

In [5]:
expr = expr.accept(DistributeMathExpr())

pprint(expr)
pprint(expr.accept(PrintMathExpr()))

MathAdd(
    class_='MathAdd',
    expr1=MathMul(
        class_='MathMul',
        expr1=MathNum(class_='MathNum', value=1),
        expr2=MathNum(class_='MathNum', value=3)
    ),
    expr2=MathMul(
        class_='MathMul',
        expr1=MathNum(class_='MathNum', value=2),
        expr2=MathNum(class_='MathNum', value=3)
    )
)

1 * 3 + 2 * 3

## Proper Order

In [6]:
expr = MathStr(string="a * (b * c)")

pprint(expr)
pprint(expr.accept(VerbosePrintMathExpr()))

MathMul(
    class_='MathMul',
    expr1=MathVar(class_='MathVar', name='a'),
    expr2=MathMul(
        class_='MathMul',
        expr1=MathVar(class_='MathVar', name='b'),
        expr2=MathVar(class_='MathVar', name='c')
    )
)

a * (b * c)

In [7]:
expr = expr.accept(ProperOrderMathExpr())

pprint(expr)
pprint(expr.accept(VerbosePrintMathExpr()))

MathMul(
    class_='MathMul',
    expr1=MathMul(
        class_='MathMul',
        expr1=MathVar(class_='MathVar', name='a'),
        expr2=MathVar(class_='MathVar', name='b')
    ),
    expr2=MathVar(class_='MathVar', name='c')
)

(a * b) * c

## Partition

In [8]:
expr = MathFunc(func="sin",expr=MathNum(value=1)) * MathImag() * MathNum(value=2) * MathVar(name="a")

pprint(expr)
pprint(expr.accept(PrintMathExpr()))

MathMul(
    class_='MathMul',
    expr1=MathMul(
        class_='MathMul',
        expr1=MathMul(
            class_='MathMul',
            expr1=MathFunc(class_='MathFunc', func='sin', expr=MathNum(class_='MathNum', value=1)),
            expr2=MathImag(class_='MathImag')
        ),
        expr2=MathNum(class_='MathNum', value=2)
    ),
    expr2=MathVar(class_='MathVar', name='a')
)

sin(1) * 1j * 2 * a

In [9]:
expr = expr.accept(PartitionMathExpr())


pprint(expr.accept(PrintMathExpr()))

expr = expr.accept(PartitionMathExpr())

pprint(expr.accept(PrintMathExpr()))

expr = expr.accept(PartitionMathExpr())

pprint(expr)
pprint(expr.accept(PrintMathExpr()))

1j * sin(1) * 2 * a

1j * 2 * sin(1) * a

MathMul(
    class_='MathMul',
    expr1=MathMul(
        class_='MathMul',
        expr1=MathMul(
            class_='MathMul',
            expr1=MathImag(class_='MathImag'),
            expr2=MathNum(class_='MathNum', value=2)
        ),
        expr2=MathVar(class_='MathVar', name='a')
    ),
    expr2=MathFunc(class_='MathFunc', func='sin', expr=MathNum(class_='MathNum', value=1))
)

1j * 2 * a * sin(1)

# Operator

In [10]:
X,Y,Z,I,A,C,J = PauliX(),PauliY(),PauliZ(),PauliI(),Annihilation(),Creation(),Identity()

In [11]:
op = MathNum(value=1) * (X+Y-Z*I) @ (A*C)

pprint(op)
pprint(op.accept(PrintOperator()))

OperatorKron(
    class_='OperatorKron',
    op1=OperatorScalarMul(
        class_='OperatorScalarMul',
        op=OperatorSub(
            class_='OperatorSub',
            op1=OperatorAdd(class_='OperatorAdd', op1=PauliX(class_='PauliX'), op2=PauliY(class_='PauliY')),
            op2=OperatorMul(class_='OperatorMul', op1=PauliZ(class_='PauliZ'), op2=PauliI(class_='PauliI'))
        ),
        expr=MathNum(class_='MathNum', value=1)
    ),
    op2=OperatorMul(
        class_='OperatorMul',
        op1=Annihilation(class_='Annihilation'),
        op2=Creation(class_='Creation')
    )
)

(1) * (PauliX() + PauliY() - PauliZ() * PauliI()) @ (Annihilation() * Creation())

## Distribute

In [12]:
op = X @ (Y+Z)

pprint(op)
pprint(op.accept(PrintOperator()))

OperatorKron(
    class_='OperatorKron',
    op1=PauliX(class_='PauliX'),
    op2=OperatorAdd(class_='OperatorAdd', op1=PauliY(class_='PauliY'), op2=PauliZ(class_='PauliZ'))
)

PauliX() @ (PauliY() + PauliZ())

In [13]:
op = op.accept(OperatorDistribute())

pprint(op)
pprint(op.accept(PrintOperator()))

OperatorAdd(
    class_='OperatorAdd',
    op1=OperatorKron(class_='OperatorKron', op1=PauliX(class_='PauliX'), op2=PauliY(class_='PauliY')),
    op2=OperatorKron(class_='OperatorKron', op1=PauliX(class_='PauliX'), op2=PauliZ(class_='PauliZ'))
)

PauliX() @ PauliY() + PauliX() @ PauliZ()

## Proper Order

In [14]:
op = X @ (X @ A)

pprint(op)
pprint(op.accept(VerbosePrintOperator()))

OperatorKron(
    class_='OperatorKron',
    op1=PauliX(class_='PauliX'),
    op2=OperatorKron(class_='OperatorKron', op1=PauliX(class_='PauliX'), op2=Annihilation(class_='Annihilation'))
)

PauliX() @ (PauliX() @ Annihilation())

In [15]:
op = op.accept(ProperOrder())

pprint(op)
pprint(op.accept(VerbosePrintOperator()))

OperatorKron(
    class_='OperatorKron',
    op1=OperatorKron(class_='OperatorKron', op1=PauliX(class_='PauliX'), op2=PauliX(class_='PauliX')),
    op2=Annihilation(class_='Annihilation')
)

(PauliX() @ PauliX()) @ Annihilation()

## Gather MathExpr

In [16]:
op = (1*X) @ (2*X) 

pprint(op)
pprint(op.accept(PrintOperator()))

OperatorKron(
    class_='OperatorKron',
    op1=OperatorScalarMul(
        class_='OperatorScalarMul',
        op=PauliX(class_='PauliX'),
        expr=MathNum(class_='MathNum', value=1)
    ),
    op2=OperatorScalarMul(
        class_='OperatorScalarMul',
        op=PauliX(class_='PauliX'),
        expr=MathNum(class_='MathNum', value=2)
    )
)

(1) * PauliX() @ (2) * PauliX()

In [17]:
op = op.accept(GatherMathExpr())

pprint(op)
pprint(op.accept(PrintOperator()))

OperatorScalarMul(
    class_='OperatorScalarMul',
    op=OperatorKron(class_='OperatorKron', op1=PauliX(class_='PauliX'), op2=PauliX(class_='PauliX')),
    expr=MathMul(
        class_='MathMul',
        expr1=MathNum(class_='MathNum', value=1),
        expr2=MathNum(class_='MathNum', value=2)
    )
)

(1 * 2) * (PauliX() @ PauliX())

## Gather Pauli

In [18]:
op = A @ X @ C @ Y

pprint(op)
pprint(op.accept(PrintOperator()))

OperatorKron(
    class_='OperatorKron',
    op1=OperatorKron(
        class_='OperatorKron',
        op1=OperatorKron(
            class_='OperatorKron',
            op1=Annihilation(class_='Annihilation'),
            op2=PauliX(class_='PauliX')
        ),
        op2=Creation(class_='Creation')
    ),
    op2=PauliY(class_='PauliY')
)

Annihilation() @ PauliX() @ Creation() @ PauliY()

In [19]:
op = op.accept(GatherPauli())
op = op.accept(GatherPauli())

pprint(op)
pprint(op.accept(PrintOperator()))

OperatorKron(
    class_='OperatorKron',
    op1=OperatorKron(
        class_='OperatorKron',
        op1=OperatorKron(
            class_='OperatorKron',
            op1=PauliX(class_='PauliX'),
            op2=Annihilation(class_='Annihilation')
        ),
        op2=PauliY(class_='PauliY')
    ),
    op2=Creation(class_='Creation')
)

PauliX() @ Annihilation() @ PauliY() @ Creation()

## Normal Order

In [20]:
op = A * C

pprint(op)
pprint(op.accept(PrintOperator()))

OperatorMul(class_='OperatorMul', op1=Annihilation(class_='Annihilation'), op2=Creation(class_='Creation'))

Annihilation() * Creation()

In [21]:
op = op.accept(NormalOrder())

pprint(op)
pprint(op.accept(PrintOperator()))

OperatorSub(
    class_='OperatorSub',
    op1=OperatorMul(
        class_='OperatorMul',
        op1=Creation(class_='Creation'),
        op2=Annihilation(class_='Annihilation')
    ),
    op2=Identity(class_='Identity')
)

Creation() * Annihilation() - Identity()

## Term Index

In [22]:
op = X @ Y @ Z @ A @ C @ (C*A)

pprint(op)
pprint(op.accept(PrintOperator()))

OperatorKron(
    class_='OperatorKron',
    op1=OperatorKron(
        class_='OperatorKron',
        op1=OperatorKron(
            class_='OperatorKron',
            op1=OperatorKron(
                class_='OperatorKron',
                op1=OperatorKron(class_='OperatorKron', op1=PauliX(class_='PauliX'), op2=PauliY(class_='PauliY')),
                op2=PauliZ(class_='PauliZ')
            ),
            op2=Annihilation(class_='Annihilation')
        ),
        op2=Creation(class_='Creation')
    ),
    op2=OperatorMul(
        class_='OperatorMul',
        op1=Creation(class_='Creation'),
        op2=Annihilation(class_='Annihilation')
    )
)

PauliX() @ PauliY() @ PauliZ() @ Annihilation() @ Creation() @ (Creation() * Annihilation())

In [23]:
pprint(op.accept(TermIndex()))

[1, 2, 3, 1, 0, 1, 1, 2, 1]

## Sorted Order

In [24]:
op = X @ C + Y @ C + X @ A

pprint(op)
pprint(op.accept(PrintOperator()))

OperatorAdd(
    class_='OperatorAdd',
    op1=OperatorAdd(
        class_='OperatorAdd',
        op1=OperatorKron(class_='OperatorKron', op1=PauliX(class_='PauliX'), op2=Creation(class_='Creation')),
        op2=OperatorKron(class_='OperatorKron', op1=PauliY(class_='PauliY'), op2=Creation(class_='Creation'))
    ),
    op2=OperatorKron(class_='OperatorKron', op1=PauliX(class_='PauliX'), op2=Annihilation(class_='Annihilation'))
)

PauliX() @ Creation() + PauliY() @ Creation() + PauliX() @ Annihilation()

In [25]:
op = op.accept(SortedOrder())

pprint(op)
pprint(op.accept(PrintOperator()))
pprint(op.accept(TermIndex()))

OperatorAdd(
    class_='OperatorAdd',
    op1=OperatorAdd(
        class_='OperatorAdd',
        op1=OperatorKron(
            class_='OperatorKron',
            op1=PauliX(class_='PauliX'),
            op2=Annihilation(class_='Annihilation')
        ),
        op2=OperatorKron(class_='OperatorKron', op1=PauliX(class_='PauliX'), op2=Creation(class_='Creation'))
    ),
    op2=OperatorKron(class_='OperatorKron', op1=PauliY(class_='PauliY'), op2=Creation(class_='Creation'))
)

PauliX() @ Annihilation() + PauliX() @ Creation() + PauliY() @ Creation()

[[1, 1, 0], [1, 1, 1], [2, 1, 1]]

## Pauli Algebra

In [26]:
op = X * Y

pprint(op)
pprint(op.accept(PrintOperator()))

OperatorMul(class_='OperatorMul', op1=PauliX(class_='PauliX'), op2=PauliY(class_='PauliY'))

PauliX() * PauliY()

In [27]:
op = op.accept(PauliAlgebra())

pprint(op)
pprint(op.accept(PrintOperator()))

OperatorScalarMul(class_='OperatorScalarMul', op=PauliZ(class_='PauliZ'), expr=MathImag(class_='MathImag'))

(1j) * PauliZ()

## Prune Identity

In [28]:
op = A * C * C * A * C

pprint(op)
pprint(op.accept(PrintOperator()))

OperatorMul(
    class_='OperatorMul',
    op1=OperatorMul(
        class_='OperatorMul',
        op1=OperatorMul(
            class_='OperatorMul',
            op1=OperatorMul(
                class_='OperatorMul',
                op1=Annihilation(class_='Annihilation'),
                op2=Creation(class_='Creation')
            ),
            op2=Creation(class_='Creation')
        ),
        op2=Annihilation(class_='Annihilation')
    ),
    op2=Creation(class_='Creation')
)

Annihilation() * Creation() * Creation() * Annihilation() * Creation()

In [29]:
op = op.accept(NormalOrder())
op = op.accept(OperatorDistribute())
op = op.accept(NormalOrder())
op = op.accept(OperatorDistribute())
op = op.accept(NormalOrder())
op = op.accept(OperatorDistribute())

pprint(op.accept(PrintOperator()))

Creation() * Creation() * Annihilation() * Creation() * Annihilation() - Creation() * Identity() * Creation() * 
Annihilation() + (-1) * (Creation() * Identity() * Creation() * Annihilation()) + (-1) * (Creation() * Creation() *
Annihilation() * Identity() - Creation() * Identity() * Identity()) - (-1) * (Creation() * Identity() * Identity())

In [30]:
op = op.accept(PruneIdentity())

pprint(op.accept(PrintOperator()))

Creation() * Creation() * Annihilation() * Creation() * Annihilation() - Creation() * Creation() * Annihilation() +
(-1) * (Creation() * Creation() * Annihilation()) + (-1) * (Creation() * Creation() * Annihilation() - Creation()) 
- (-1) * Creation()

# Example

In [31]:
def repeat(model: Union[MathExpr, Operator], visitor: List[Visitor], verbose=True):
    i = 0
    while True:
        _model = model.accept(visitor())
        try:
            model.accept(VerifyHilbertSpace())
        except:
            raise Exception("{}".format(visitor.__name__))
        if _model == model:
            break
        i += 1
        model = _model

        if verbose:
            pprint(
                "\n{:^20}:".format(visitor.__name__ + str(i)),
                "\n" + model.accept(VerbosePrintOperator()),
            )

    print("Ran {} {} times".format(visitor.__name__, i))
    return model


def multivisitor(
    model: Union[MathExpr, Operator], visitors: List[Visitor], verbose=True
):
    for visitor in visitors:
        model = repeat(model, visitor, verbose)
    return model

In [32]:
op = X @ C @ (X*Y) @ (A * C * C * A * C * C) @ (X @ X @ A @ C)

pprint(op.accept(PrintOperator()))

PauliX() @ Creation() @ (PauliX() * PauliY()) @ (Annihilation() * Creation() * Creation() * Annihilation() * 
Creation() * Creation()) @ PauliX() @ PauliX() @ Annihilation() @ Creation()

In [35]:
print("=" * 40)
while True:
    _op = multivisitor(
        op,
        [
            OperatorDistribute,
            GatherMathExpr,
            ProperOrder,
            PauliAlgebra,
            GatherMathExpr,
            GatherPauli,
            NormalOrder,
            OperatorDistribute,
            GatherMathExpr,
            ProperOrder,
            PauliAlgebra,
            GatherMathExpr,
            GatherPauli,
            SortedOrder,
            PruneIdentity,
            DistributeMathExpr,
            ProperOrderMathExpr,
            PartitionMathExpr,
        ],
        verbose=False,
    )
    print("=" * 40)
    if op == _op:
        break
    op = _op

pprint(op.accept(PrintOperator()))

Ran OperatorDistribute 0 times
Ran GatherMathExpr 0 times
Ran ProperOrder 0 times
Ran PauliAlgebra 0 times
Ran GatherMathExpr 0 times
Ran GatherPauli 0 times
Ran NormalOrder 0 times
Ran OperatorDistribute 0 times
Ran GatherMathExpr 0 times
Ran ProperOrder 0 times
Ran PauliAlgebra 0 times
Ran GatherMathExpr 0 times
Ran GatherPauli 0 times
Ran SortedOrder 0 times
Ran PruneIdentity 0 times
Ran DistributeMathExpr 0 times
Ran ProperOrderMathExpr 0 times
Ran PartitionMathExpr 0 times


(1j * -1 * -1 + 1j * -1 * -1 + 1j * -1 * -1 + 1j * -1 * -1 + 1j * -1 * -1 + 1j * -1 * -1) * (PauliX() @ PauliX() @ 
PauliX() @ PauliZ() @ Creation() @ (Creation() * Creation()) @ Annihilation() @ Creation()) + (1j * -1 + 1j * -1 + 
1j * -1 + 1j * -1 + 1j * -1 + 1j * -1) * (PauliX() @ PauliX() @ PauliX() @ PauliZ() @ Creation() @ (Creation() * 
Creation() * Creation() * Annihilation()) @ Annihilation() @ Creation()) + (1j) * (PauliX() @ PauliX() @ PauliX() @
PauliZ() @ Creation() @ (Creation() * Creation() * Creation() * Creation() * Annihilation() * Annihilation()) @ 
Annihilation() @ Creation())